In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['figure.dpi'] = 300

In [ ]:
import os
os.environ['PATH'] = "/usr/local/sbin:/usr/local/bin:/usr/sbin:/opt/R/4.2.3/bin:/usr/bin:/sbin:/bin:/opt/miniconda3/conda/bin:/opt/miniconda3/bin"
%load_ext rpy2.ipython

In [ ]:
%%R
library(magrittr)
library("data.table")
devtools::load_all('~/libs/genesets')
source("https://raw.githubusercontent.com/slagtermaarten/TONIC/refs/heads/master/R/GSEA_funcs_paired.R")

In [ ]:
%%R
devtools::load_all('~/libs/maartenutils')

# Hallmarks Baseline Onnivo

In [ ]:
res_p = pd.read_csv('../../processed_data/flexgsea_new_cohorts_hallmarks_on_nivo_Monday.tsv', sep='\t', index_col=0)

In [ ]:
%R -i res_p

In [ ]:
%%R
caplist_def <- c('DNA',
 'G2-M',
 'IL-2/STAT5',
 'IL-6/JAK/STAT3',
 'KRAS',
 'PI3K/AKT/mTOR',
 'TGF',
 'TNFA',
                 'NFKB',
 'UV',
 'mTORC1',
 'p53', 'JAK')
caplist_tonic <- c(caplist_def, 'Gu-Trantien', 'IFNy', 'IFNg', 'TNBC', 'TIL',
                   'mm2', 'Ichao1', 'cells', 'cell', 'TIS', 'score', 's', 'ng',
                   'LDH', 'IFN', 'Th1', 'CRP', 'TIL', 'CIBERSORT', 'GC',
                   'Gu-Trantien',
                   'gamma', 'LumA', 'LumB', 'APM', 'FastQ',
                   'ICB', 'CAF', 'TAM', 'MDSC', 'PMN', 'MO',
                   'PD-L1', 'MMR', 'loss', 'SASP', 'RNA')
if (exists('rna_read_counts_salmon')) {
  caplist_tonic %<>% { c(., rownames(rna_read_counts_salmon)) }
}
# simple_cap('ifny', caplist = caplist_tonic)
tonic_cap <- function(x, ...) {
  stopifnot(is.character(x) || is.factor(x))
  ## First some preprocessing before final capitalisation
  x <- gsub('HALLMARK_', '', x) %>% 
    gsub('-', '_', .) %>%
       gsub('\\.', ' ', .) %>%
       gsub('IFNG', 'IFNy', .) %>%
       gsub('ifn gamma', 'IFNy', .) %>%
       gsub('fqreads', 'FastQ reads', .) %>%
       gsub('GU_TRANTIEN', 'Gu-Trantien', .) %>%
       gsub('PD_L1', 'PD-L1', .) %>%
       gsub('pdl1', 'PD-L1', .)
  return(simple_cap(tolower(x), caplist = caplist_tonic, ...))
}
cor_cap <- tonic_cap(res_p$GeneSet)

In [ ]:
%R -o cor_cap
res_p.GeneSet = cor_cap

In [ ]:
order_arm = ['Doxorubicin', 'Cisplatin', 'No induction']
order_sig = ['Apical Junction',
 'Epithelial Mesenchymal Transition',
 'Angiogenesis',   
  'UV Response Dn',   
  'Myogenesis',  
  'P53 Pathway',
 'Hypoxia',
  'KRAS Signaling Dn',
 'Wnt Beta Catenin Signaling',
  'TGF Beta Signaling',
 'Estrogen Response Early',
 'Hedgehog Signaling',
 'Androgen Response',
 'Notch Signaling',
 'Apical Surface',
 'Interferon gamma Response',
  'Xenobiotic Metabolism',
  'Apoptosis',
 'Coagulation',
  'IL6 JAK STAT3 Signaling',
 'KRAS Signaling Up',
  'Allograft Rejection',
  'Complement',
 'IL2 STAT5 Signaling',
 'Heme Metabolism',
  'Pancreas Beta cells',
 'Interferon Alpha Response',
  'PI3K Akt Mtor Signaling',
 'UV Response Up',
 'Inflammatory Response',
 'TNFA Signaling Via NFKB',
 'DNA Repair',
 'Spermatogenesis',
 'Oxidative Phosphorylation', 
 'Reactive Oxigen Species Pathway', 
 'Adipogenesis',
 'Peroxisome',
 'Bile Acid Metabolism',
 'Fatty Acid Metabolism',
 'Estrogen Response Late',
 'Mitotic Spindle', 
 'E2F Targets',
'G2M Checkpoint',
 'Myc Targets V1',
 'Myc Targets V2',
 'MTORC1 Signaling',
'Cholesterol Homeostasis',
'Glycolysis',
'Protein Secretion',
'Unfolded Protein Response']
gsea_result = pd.pivot_table(res_p, index = 'arm', columns = 'GeneSet', values = 'nes').loc[order_arm, order_sig]
gsea_fdr = pd.pivot_table(res_p, index = 'arm', columns = 'GeneSet', values = 'fdr').loc[order_arm, order_sig]

In [ ]:
gsea_pv = pd.pivot_table(res_p, index = 'arm', columns = 'GeneSet', values = 'p').loc[order_arm, order_sig]

In [ ]:
bold_list = ['Interferon gamma Response', 'IL6 JAK STAT3 Signaling','Allograft Rejection',
  'Complement',
 'IL2 STAT5 Signaling', 'Interferon Alpha Response', 'Inflammatory Response',
 'TNFA Signaling Via NFKB']

In [ ]:
fdr = (gsea_fdr < 0.25).replace({True: "•", False: ''})
pv = (gsea_pv < 0.05).replace({True: "•", False: ''})

In [ ]:
combined_annot = pd.DataFrame('', index=fdr.index, columns=fdr.columns)
text_colors = pd.DataFrame('', index=fdr.index, columns=fdr.columns)

for i in range(fdr.shape[0]):
    for j in range(fdr.shape[1]):
        if fdr.iloc[i,j] == '•':
            combined_annot.iloc[i, j] = '•'
            text_colors.iloc[i,j] = 'black'
        elif pv.iloc[i,j] == '•':
            combined_annot.iloc[i, j] = '•'
            text_colors.iloc[i,j] = '#858181'
        else:
            combined_annot.iloc[i, j] = ''
            text_colors.iloc[i,j] = 'red'

In [ ]:
mpl.rcParams['font.size'] = 5
ax = sns.heatmap(gsea_result, vmin=-2, vmax=2, center=0, cmap=sns.color_palette("vlag", as_cmap=True), square=True, cbar=True, cbar_kws={'shrink': 0.2, 'anchor': (0.0, 0.43), 'aspect': 12, 'pad': 0.025},
                xticklabels=True, yticklabels=True, annot = False, fmt='', annot_kws = {'color': 'black', 'fontsize': 8})


for i in range(fdr.shape[0]):
    for j in range(fdr.shape[1]):
        text = combined_annot.iloc[i, j]
        if text:
            ax.text(j + 0.5, i + 0.5, text, ha='center', va='center',
                    color=text_colors.iloc[i, j], fontsize=8)


ax.set_yticklabels(labels = order_arm, size = 5)

ax.set_xticklabels(order_sig, fontdict={'fontsize': 5})
ax.set(xlabel=None)
ax.set(ylabel=None)
for lab in ax.get_xticklabels():
    text =  lab.get_text()
    if text in bold_list: # lets highlight row 2
        # set the properties of the ticklabel
        lab.set_weight('bold')
cax = ax.figure.axes[-1]
cax.tick_params(width=0.5, length=3)
ax.tick_params(width=0.6, length=3)
plt.savefig("Fig3C_hallmarks_onNivo.pdf", format="pdf", bbox_inches = 'tight')

# TME signatures (Bagaev) Baseline On-nivo

In [ ]:
res_doxo_on = pd.read_csv('../../processed_data/flexgsea_new_cohorts_bagaev_dox_onnivo.tsv', sep='\t', index_col=0)
res_cis_on = pd.read_csv('../../processed_data/flexgsea_new_cohorts_bagaev_cis_onnivo.tsv', sep='\t', index_col=0)
res_noi_on = pd.read_csv('../../processed_data/flexgsea_new_cohorts_bagaev_noi_onnivo.tsv', sep='\t', index_col=0)
res_on = pd.concat([res_doxo_on, res_cis_on, res_noi_on])

In [ ]:
order_arm = ['Doxorubicin', 'Cisplatin', 'No induction']
gsea_result = pd.pivot_table(res_on, index = 'arm', columns = 'GeneSet', values = 'nes').loc[order_arm]
gsea_fdr = pd.pivot_table(res_on, index = 'arm', columns = 'GeneSet', values = 'fdr').loc[order_arm]
gsea_pv = pd.pivot_table(res_on, index = 'arm', columns = 'GeneSet', values = 'p').loc[order_arm]

In [ ]:
gsea_result.columns = gsea_result.columns.str.replace('_', ' ')
gsea_fdr.columns = gsea_fdr.columns.str.replace('_', ' ')
gsea_pv.columns = gsea_pv.columns.str.replace('_', ' ')

In [ ]:
fdr = (gsea_fdr < 0.25).replace({True: "•", False: ''})
pv = (gsea_pv < 0.05).replace({True: "•", False: ''})

combined_annot = pd.DataFrame('', index=fdr.index, columns=fdr.columns)
text_colors = pd.DataFrame('', index=fdr.index, columns=fdr.columns)

for i in range(fdr.shape[0]):
    for j in range(fdr.shape[1]):
        if fdr.iloc[i,j] == '•':
            combined_annot.iloc[i, j] = '•'
            text_colors.iloc[i,j] = 'black'
        elif pv.iloc[i,j] == '•':
            combined_annot.iloc[i, j] = '•'
            text_colors.iloc[i,j] = '#858181'
        else:
            combined_annot.iloc[i, j] = ''
            text_colors.iloc[i,j] = 'red'

In [ ]:
#plt.rcParams.update({'font.size': 5})
mpl.rcParams['font.size'] = 5
fig, ax = plt.subplots(1,1, figsize = (4,4))
_ = sns.heatmap(gsea_result, vmin=-2, vmax=2, center=0, cmap=sns.color_palette("vlag", as_cmap=True), square=True, cbar=True, cbar_kws={'shrink': 0.2, 'anchor': (0.0, 0.43), 'aspect': 12},
                xticklabels=True, yticklabels=True, annot = False, fmt='', ax=ax, annot_kws = {'color': 'black', 'fontsize': 8})

for i in range(fdr.shape[0]):
    for j in range(fdr.shape[1]):
        text = combined_annot.iloc[i, j]
        if text:
            ax.text(j + 0.5, i + 0.5, text, ha='center', va='center',
                    color=text_colors.iloc[i, j], fontsize=8)

ax.set_yticklabels(labels = order_arm, size = 5)

ax.set_xticklabels(ax.get_xticklabels(), fontdict={'fontsize': 5})
ax.set(xlabel=None)
ax.set(ylabel=None)
cax = ax.figure.axes[-1]
cax.tick_params(width=0.5, length=3)
ax.tick_params(width=0.6, length=3)
plt.savefig("Fig3D_Bagaev_onNivo.pdf", format="pdf", bbox_inches = 'tight')